# Imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import requests

In [8]:
driver = webdriver.Chrome()
driver.get('https://portal.gupy.io/')
driver.maximize_window()
try:
    cuidado_golpes = driver.find_element(By.XPATH, '//*[@id="radix-0"]/div[2]/button')
    cuidado_golpes.click()
except:
    pass
search = driver.find_element(By.XPATH, '/html/body/div/main/div/div/div[2]/div/div/div/div/div[2]/input')
search.send_keys('analista de dados')
search.send_keys(Keys.ENTER)

SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [9]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

lista_de_vagas = soup.find_all('div', {'class':'sc-a3bd7ea-0 HCzvP'})

df_vagas = pd.DataFrame(
        columns = ['site_da_vaga',
        'link',
        'data_publicacao',
        'data_coleta',
        'titulo_da_vaga',
        'local',
        'modalidade',
        'contrato',
        'acessibilidade',
        'nome_empresa',
        'codigo_vaga',
        'descricao']
    )

for vaga in lista_de_vagas:
# vaga = lista_de_vagas[0]
    df_aux = pd.DataFrame(
        columns = ['site_da_vaga',
        'link',
        'data_publicacao',
        'data_coleta',
        'titulo_da_vaga',
        'local',
        'regime',
        'modalidade',
        'contrato',
        'acessibilidade',
        'nome_empresa',
        'codigo_vaga',
        'descricao']
    )

    df_aux.loc[0,'site_da_vaga'] = 'Gupy'
    df_aux.loc[0,'data_coleta'] = datetime.datetime.today().strftime('%Y-%m-%d')
    df_aux.loc[0,'link'] = vaga.findAll('a')[0]['href']
    df_aux.loc[0,'titulo_da_vaga'] = vaga.findAll('h2')[0].text
    try: 
        df_aux.loc[0,'local'] = vaga.findAll('span', {'class',"sc-23336bc7-1 cezNaf"})[0].text
    except:
        df_aux.loc[0,'local'] = np.nan
    try:
        df_aux.loc[0,'modalidade'] = vaga.findAll('span', {'class',"sc-23336bc7-1 cezNaf"})[1].text
    except:
        df_aux.loc[0,'modalidade'] = np.nan
    try:
        df_aux.loc[0,'contrato'] = vaga.findAll('span', {'class',"sc-23336bc7-1 cezNaf"})[2].text
    except:
        df_aux.loc[0,'contrato'] = np.nan
    try:
        df_aux.loc[0,'acessibilidade'] = vaga.findAll('span', {'class',"sc-23336bc7-1 cezNaf"})[3].text
    except:
        df_aux.loc[0,'acessibilidade'] = np.nan

    df_aux.loc[0,'nome_empresa'] = vaga.findAll('p', {'class':'sc-dPyBCJ kyoAxx sc-a3bd7ea-6 cQyvth'})[0].text
    df_aux.loc[0,'data_publicacao'] = vaga.findAll('p', {'class':'sc-dPyBCJ kyoAxx sc-1db88588-0 inqtnx'})[0].text

    try:
        response = requests.get(vaga.findAll('a')[0]['href']) # link da vaga
        page = BeautifulSoup(response.text)
        lista_descricao_em_texto = [page.findAll('div', {'data-testid':'text-section'})[i].text for i in range(len(page.findAll('div', {'data-testid':'text-section'})))]
        descricao_completa = '\n'.join(lista_descricao_em_texto)
        df_aux.loc[0,'descricao'] = descricao_completa
    except:
        df_aux.loc[0,'descricao'] = np.nan

    df_vagas = pd.concat([df_vagas, df_aux], axis=0, ignore_index=True)
    
df_vagas = df_vagas.reset_index(drop=True)



In [10]:
df_vagas

,site_da_vaga,link,data_publicacao,data_coleta,titulo_da_vaga,local,modalidade,contrato,acessibilidade,nome_empresa,codigo_vaga,descricao,regime
0,Gupy,https://vtal.gupy.io/job/eyJqb2JJZCI6NjU4NzUyN...,Publicada em: 19/01/2024,2024-01-20,Analista de Dados II,Rio de Jan... - RJ,Presencial,Efetivo,Também p/ PcD,V.tal | o futuro passa por aqui,NaN,Descrição da vagaO profissional fará parte do ...,NaN
1,Gupy,https://celcoin.gupy.io/job/eyJqb2JJZCI6NjIyOD...,Publicada em: 19/01/2024,2024-01-20,Analista de Dados Jr.,Barueri - SP,Híbrido,Efetivo,Também p/ PcD,Celcoin,NaN,Descrição da vagaFaça parte de uma Startup ino...,NaN
2,Gupy,https://gruporpn.gupy.io/job/eyJqb2JJZCI6NjY0O...,Publicada em: 19/01/2024,2024-01-20,Grupo RPN | Analista de Inteligência de Dados ...,Americana - SP,Presencial,Efetivo,Também p/ PcD,Grupo RPN,NaN,Descrição da vagaA pessoa Analista de Inteligê...,NaN
3,Gupy,https://stefanini.gupy.io/job/eyJqb2JJZCI6NjY0...,Publicada em: 18/01/2024,2024-01-20,ANALISTA DADOS SR,Não informado,Remoto,Efetivo,Também p/ PcD,Stefanini Group,NaN,Descrição da vagaSomos a maior multinacional b...,NaN
4,Gupy,https://jbs.gupy.io/job/eyJqb2JJZCI6NjY0MDcxOC...,Publicada em: 18/01/2024,2024-01-20,Analista de Planejamento - Análise de Dados - ...,São Paulo - SP,Presencial,Efetivo,NaN,JBS,NaN,Descrição da vagaA JBS é a maior empresa de al...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Gupy,https://infotecbrasil.gupy.io/job/eyJqb2JJZCI6...,Publicada em: 26/12/2022,2024-01-20,Analista de Dados - Home Office,Rio de Jan... - RJ,Remoto,Efetivo,Também p/ PcD,Infotec Brasil,NaN,Descrição da vagaExecutar atividades de suport...,NaN
192,Gupy,https://gruposkytech&76015&inactive.gupy.io/jo...,Publicada em: 31/10/2022,2024-01-20,Analista de dados,São Paulo - SP,Presencial,Efetivo,NaN,Grupo Skytech,NaN,NaN,NaN
193,Gupy,https://brivia.gupy.io/job/eyJqb2JJZCI6OTQ5ODA...,Publicada em: 08/06/2021,2024-01-20,Analista de Dados,Rio de Jan... - RJ,Remoto,Banco de Talentos,Também p/ PcD,Brivia,NaN,Descrição da vagaVaga para nosso BANCO DE TALE...,NaN
194,Gupy,https://queroserglobalsys.gupy.io/job/eyJqb2JJ...,Publicada em: 04/06/2021,2024-01-20,Banco de Talentos - Analista de Dados/ Analist...,Não informado,Banco de Talentos,Também p/ PcD,NaN,Globalsys Soluções em TI,NaN,,NaN


In [11]:
df_vagas.to_excel('../data_raw/vagas_gupy.xlsx', index=False)

## ---------------------------------